In [ ]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from dosipy.constants import eps_0
from dosipy.utils.integrate import elementwise_quad, elementwise_rectquad

from utils import fraunhofer_distance, reflection_coefficient

In [ ]:
sns.set_theme(style='ticks',
              font_scale=1.5,
              palette='colorblind',
              rc={'text.usetex': True,
                  'text.latex.preamble': r'\usepackage{amsmath}',
                  'font.family': 'serif'})

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# working frequency, Hz
f = np.array([10, 30, 90]) * 1e9

# distance from the antenna
d_10_30 = np.array([5, 10, 15]) / 1000  # meters
d_90 = np.array([2, 5, 10]) / 1000  # meters

# dry skin density, kg/m3
rho = 1109

# conductivity, S/m
sigma = np.array([8.48, 27.31, 41.94])

# dielectric constant
eps_r = np.array([32.41, 16.63, 6.83])

# reflection coefficient
eps_i = sigma / (2 * np.pi * f * eps_0)
eps = eps_r - 1j * eps_i
gamma = reflection_coefficient(eps)

# power transmission coefficient
T_tr = 1 - gamma ** 2

# exposed surface extent
exposure_extent = (0.02, 0.02)  # meters x meters

# integration area
A_4 = exposure_extent[0] * exposure_extent[1]  # meters squared
A_1 = A_4 / 4  # meters squared

# exposed surface coordinates
xt = np.linspace(-exposure_extent[0]/2, exposure_extent[0]/2)
yt = np.linspace(-exposure_extent[1]/2, exposure_extent[1]/2)

In [ ]:
# power density distribution on the exposed surface
PDinc = np.load(os.path.join('data', '02_power_density_d10mm_f90GHz.npy'))

In [ ]:
# averaged total incident power density on the surface
PDinc_tot = np.linalg.norm(PDinc.real, ord=2, axis=2)

In [ ]:
sPDinc_tot_4 = 1 / (2 * A_4) * elementwise_rectquad(xt, yt, PDinc_tot)
sPDinc_tot_1 = 1 / (2 * A_1) * elementwise_rectquad(xt, yt, PDinc_tot)
sPDinc_tot_4, sPDinc_tot_1

In [ ]:
# averaged total incident power density on the surface
PDinc_n = -PDinc[:, :, 2].real

In [ ]:
sPDinc_n_4 = 1 / (2 * A_4) * elementwise_rectquad(xt, yt, PDinc_n)
sPDinc_n_1 = 1 / (2 * A_1) * elementwise_rectquad(xt, yt, PDinc_n)
sPDinc_n_4, sPDinc_n_1

In [ ]:
# visualize
fig, ax = plt.subplots()
xmin, xmax = xt.min(), xt.max()
ymin, ymax = yt.min(), yt.max()
bbox = [xmin, xmax, ymin, ymax]
cm = ax.imshow(PDinc_n,
               extent=bbox,
               origin='lower',
               interpolation='bicubic',
               cmap='viridis')
cb = fig.colorbar(cm, ax=ax,
                  label='power density (W$/$m$^3$)')
ax.set(xlabel='$x$ (cm)',
       ylabel='$y$ (cm)',
       xticks=[xmin, 0, xmax],
       yticks=[ymin, 0, ymax],
       xticklabels=[xmin * 100, 0, xmax * 100],
       yticklabels=[ymin * 100, 0, ymax * 100])
sns.despine(fig=fig, ax=ax)

# Spatially averaged abosorbed power density

In [ ]:
sPDab_1 = T_tr[2] * sPDinc_n_1
sPDab_1